In [1]:
import random
import torch

from utils.Logger import showPlot
from utils.Timer import asMinutes, timeSince

%load_ext autoreload
from data.exampleJP_Data import Lang, prepareData
from model.rnn_model.encoderRNN import EncoderGRU, EncoderLSTM
from model.rnn_model.decoderRNN import (
    DecoderGRU, DecoderLSTM, 
    AttnDecoderGRU, AttnDecoderLSTM1, AttnDecoderLSTM2
)
from model.seq2seq_Model import (
    Seq2Seq_batch_ptModel, 
    Seq2SeqTranslate_ptTokenizer,
)
from baselineJP_ExpTrain import example_ExpTrain
from baseline_ExpEval import evaluate_batch_randomly
%autoreload


DATA_DIR = '_data_example'

## load data

In [2]:
if True:    # 解凍する場合は True に書き換え
    !mkdir  $DATA_DIR
    %cd  $DATA_DIR

    !git clone https://github.com/odashi/small_parallel_enja
    !echo '------------------'
    !head -n 5 small_parallel_enja/train.en
    !head -n 5 small_parallel_enja/train.ja
    !echo '------------------'

    !ls
    %cd ../

mkdir: _data_example: File exists
/Volumes/GoogleDrive/マイドライブ/__datasets__/[CV][NLP]「センター試験xml」/annotate_img/datas/center_exam_2011/construct/Experiment/_data_example
fatal: destination path 'small_parallel_enja' already exists and is not an empty directory.
------------------
i can 't tell who will arrive first .
many animals have been destroyed by men .
i 'm in the tennis club .
emi looks happy .
please bear this fact in mind .
誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
私 は テニス 部員 で す 。
エミ は 幸せ そう に 見え ま す 。
この 事実 を 心 に 留め て お い て 下さ い 。
------------------
data                data.zip            small_parallel_enja
/Volumes/GoogleDrive/マイドライブ/__datasets__/[CV][NLP]「センター試験xml」/annotate_img/datas/center_exam_2011/construct/Experiment


## setup Experiment

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## setup Data

In [4]:
input_lang, output_lang, pairs = prepareData(
    'en', 'ja', f'{DATA_DIR}/small_parallel_enja', False)

Reading lines...
Read 50000 sentence pairs
Counting words...
Counted words:
en 6637
ja 8777


In [5]:
print(random.choice(pairs))
print(input_lang.n_words, output_lang.n_words)

['he always wants to have his own way .', '彼 は いつ で も 自分 勝手 に や り たが る 。']
6637 8777


In [6]:
# train / test split
from sklearn.model_selection import train_test_split
train_pairs, test_pairs = train_test_split(pairs, test_size=0.2)

## setup Model

In [7]:
## Model
tokenizer = Seq2SeqTranslate_ptTokenizer(
                    input_lang, output_lang, device)

### test

In [8]:
batch_size = 10
emb_size = 8
hid_size = 12
MAX_LENGTH = 18

test_encoder = EncoderLSTM(input_lang.n_words, emb_size, hid_size)
test_decoder1 = AttnDecoderLSTM1(
                    emb_size, hid_size, output_lang.n_words,
                    device, max_length=MAX_LENGTH)
seq2seq_test_model = Seq2Seq_batch_ptModel(
                    tokenizer, device,
                    dropout_p=0.1, max_length=MAX_LENGTH)
seq2seq_test_model.set_models(test_encoder, test_decoder1)
seq2seq_test_model.exec_test(train_pairs, batch_size=batch_size)


[Info] input_batch.shape, input_lens.shape
     =  torch.Size([13, 10]) torch.Size([10])
[Info] enc_outputs.shape, hidden_h.shape, hidden_c.shape
     =  torch.Size([13, 10, 24]) torch.Size([1, 10, 12]) torch.Size([1, 10, 12])
[Info] hidden[0].shape, hidden[1].shape
     =  torch.Size([10, 12]) torch.Size([10, 12])
[Info] dec_outputs.shape, hidden[0].shape, hidden[1].shape, attn_weights.shape
     =  torch.Size([10, 8777]) torch.Size([10, 12]) torch.Size([10, 12]) torch.Size([10, 18])
[Info] loss.item()
     =  9.08955192565918


In [9]:
attn_size = 9

test_decoder2 = AttnDecoderLSTM2(
                    emb_size, hid_size, attn_size, 
                    output_lang.n_words, device).to(device)
seq2seq_test_model.set_models(test_encoder, test_decoder2)
seq2seq_test_model.exec_test(train_pairs, batch_size=batch_size)

[Info] input_batch.shape, input_lens.shape
     =  torch.Size([13, 10]) torch.Size([10])
[Info] enc_outputs.shape, hidden_h.shape, hidden_c.shape
     =  torch.Size([13, 10, 24]) torch.Size([1, 10, 12]) torch.Size([1, 10, 12])
[Info] hidden[0].shape, hidden[1].shape
     =  torch.Size([10, 12]) torch.Size([10, 12])
[Info] dec_outputs.shape, hidden[0].shape, hidden[1].shape, attn_weights.shape
     =  torch.Size([10, 8777]) torch.Size([10, 12]) torch.Size([10, 12]) torch.Size([10, 13])
[Info] loss.item()
     =  9.074653625488281


## setup Model

In [10]:
%load_ext autoreload
from model.rnn_model.decoderRNN import (
    DecoderGRU, DecoderLSTM, 
    AttnDecoderGRU, AttnDecoderLSTM1, AttnDecoderLSTM2
)
from model.seq2seq_Model import (
    Seq2Seq_batch_ptModel, 
    Seq2SeqTranslate_ptTokenizer,
)
from baselineJP_ExpTrain import example_ExpTrain
from baselineJP_ExpTrain import example_ExpTrain
%autoreload

load_model_dir = '_best_weight' if torch.cuda.is_available() else "_best_weight_CPU"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
emb_size = 1024
hidden_size = 1024

MAX_LENGTH = 18

def build_encoder():
    encoder = EncoderLSTM(input_lang.n_words, emb_size, hidden_size)
    # encoder.load_weights(
    #             load_m_dir=f'_logs/{load_model_dir}', 
    #             load_m_file_name='encoder.pth')
    return encoder

In [12]:
# Experiment
exp_train = example_ExpTrain(train_pairs, test_pairs)

## exec

In [13]:
# encoder
encoder = build_encoder()
# decoder
decoder1 = AttnDecoderLSTM1(
                emb_size, hidden_size, output_lang.n_words, device)
# decoder1.load_weights(
#             load_m_dir=f'_logs/{load_model_dir}', 
#             load_m_file_name='decoder_LSTM1.pth')

# seq2seq model
seq2seq_lstm1_model = Seq2Seq_batch_ptModel(
                        tokenizer, device,
                        dropout_p=0.1, max_length=MAX_LENGTH,
                        save_m_dir='_logs', 
                        save_m_file_names=(
                            'encoder.pth', 'decoder_LSTM1.pth'))
seq2seq_lstm1_model.set_models(encoder, decoder1)

## exec
exp_train.exec(seq2seq_lstm1_model, 
                # epochs=3, batch_size=300, 
                epochs=20, batch_size=200, 
                teacher_forcing=0.9, early_stopping=3)

2it [00:29, 14.81s/it]


KeyboardInterrupt: 

In [ ]:
# encoder
encoder = build_encoder()
# decoder
attn_size = 1024
decoder2 = AttnDecoderLSTM2(
                emb_size, hidden_size, attn_size, 
                output_lang.n_words, device)
# decoder2.load_weights(
#             load_m_dir=f'_logs/{load_model_dir}', 
#             load_m_file_name='decoder_LSTM2.pth')

# seq2seq model
seq2seq_lstm2_model = Seq2Seq_batch_ptModel(
                        tokenizer, device,
                        dropout_p=0.1, max_length=MAX_LENGTH,
                        save_m_dir='_logs', 
                        save_m_file_names=(
                            'encoder.pth', 'decoder_LSTM2.pth'))
seq2seq_lstm2_model.set_models(encoder, decoder2)

## exec
exp_train.exec(seq2seq_lstm2_model, 
                # epochs=3, batch_size=300, 
                epochs=20, batch_size=200, 
                teacher_forcing=0.9, early_stopping=3)

In [ ]:
evaluate_batch_randomly(seq2seq_lstm2_model, test_pairs, n=10)